In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json  # Importer le module json pour la sérialisation
import time
import os

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import json
import os
import time

url = "https://u.gg/lol/tier-list"

# Initialisation du navigateur avec ChromeDriverManager
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

driver.get(url)

# Attendre que le bouton "Consent" soit cliquable et cliquer dessus
wait = WebDriverWait(driver, 10)
consent_button = wait.until(EC.element_to_be_clickable((By.XPATH, "//p[contains(text(), 'Consent')]")))
consent_button.click()

file_path = "Tierlist.json"  # Chemin du fichier JSON
MAX_WAIT = 10  # Configuration du temps d'attente maximal pour WebDriverWait

# Structure pour stocker les données
data_export = []

try:
    wait = WebDriverWait(driver, MAX_WAIT)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.rt-tr-group")))

    while True:
        # Scroll en bas de la page puis en haut
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")
        time.sleep(1)

        rows = driver.find_elements(By.CSS_SELECTOR, "div.rt-tr-group")
        if not rows or (data_export and len(rows) == len(data_export)):
            break

        for i in range(len(data_export), len(rows)):
            row = rows[i]
            champion = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(3)").get_attribute("textContent")
            win_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(5)").text.strip()
            ban_rate = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(7)").text.strip()
            matches = row.find_element(By.CSS_SELECTOR, "div.rt-td:nth-of-type(9)").text.strip()
            role = row.find_element(By.CSS_SELECTOR, "img.tier-list-role").get_attribute("alt")

            # Stocker chaque champion et ses données dans un dictionnaire
            champion_data = {
                'Champion Name': champion,
                'Win rate': win_rate,
                'Ban Rate': ban_rate,
                'Matches': matches,
                'Role': role.upper()
            }
            data_export.append(champion_data)

finally:
    driver.quit()

    # Enregistrement des données dans un fichier JSON
    dos = os.path.join(os.getcwd(), 'data')  # Assurez-vous que le répertoire 'data' existe
    if not os.path.exists(dos):
        os.makedirs(dos)
    chemin_complet = os.path.join(dos, 'data_export.json')

    with open(chemin_complet, 'w', encoding='utf-8') as jsonfile:
        json.dump(data_export, jsonfile, indent=4)  # Utiliser indent pour un formatage plus lisible
